In [10]:
# pip install cvxpy # install this on first try

import ot
import pandas as pd

import numpy as np
from numpy import random
import os
import glob

import matplotlib.pylab as plt
import ot.plot
import cvxpy as cp

import scipy.stats as stats
import seaborn as sns
import scipy.special as sps
import time as t

In [26]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
files_list = glob.glob('*.{}'.format("csv"))

files_names = [s.replace(".csv", "") for s in files_list]
files_names

['SC',
 'FL',
 'TN',
 'full data',
 'NC',
 'WY',
 'GA',
 'MS',
 'AL',
 'WI',
 'MT',
 'SD',
 'KS',
 'TX']

In [23]:
files_dict = {}

for k in range(len(csv_files)):
    
    key = files_names[k]
    
    # read the csv file
    df = pd.read_csv(csv_files[k])
    
    display(df)
    
    # store in dictionary
    files_dict[key] = df

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,1115597,72,75,2010011155971,45,20245,1,1,...,6,4125,2,3,5,3,67100,67100,4500000.0,45
1,2010,201001,1115598,201,244,2010011155981,45,120045,1,1,...,6,9272,2,3,5,3,34000,34000,8296000.0,45
2,2010,201001,1115599,252,60,2010011155991,45,10045,1,1,...,6,2400,2,3,5,3,41600,79600,2412000.0,45
3,2010,201001,1115599,252,60,2010011155991,45,10045,1,2,...,6,2360,2,3,5,3,30900,79600,1823100.0,45
4,2010,201001,1115600,322,63,2010011156001,45,230045,1,1,...,6,2520,2,3,5,3,20000,22500,1260000.0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107900,2016,201601,1118795,1520859,12,2016011187951,45,60145,1,1,...,6,520,2,3,5,3,50000,74000,650000.0,45
107901,2016,201601,1118796,1520890,62,2016011187961,45,20045,1,1,...,6,2480,2,3,5,3,120050,148050,7440000.0,45
107902,2016,201601,1118796,1520890,62,2016011187961,45,20045,1,2,...,2,1650,2,3,5,3,28000,148050,1188000.0,45
107903,2016,201601,1118797,1521022,53,2016011187971,45,130045,1,1,...,6,2160,2,3,5,3,24000,24000,1296000.0,45


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,262134,61,67,2010002621341,12,80012,1,1,...,1,2211,2,3,5,3,1300,39400,87100.0,12
1,2010,201001,262134,61,67,2010002621341,12,80012,1,2,...,6,3600,2,3,5,3,30000,39400,2700000.0,12
2,2010,201001,262137,121,73,2010002621371,12,350512,1,1,...,6,2960,2,1,5,3,17000,17000,666000.0,12
3,2010,201001,262139,132,82,2010002621391,12,260512,1,1,...,2,3690,1,2,4,1,47800,87800,3280000.0,0
4,2010,201001,262139,132,82,2010002621391,12,260512,1,2,...,6,3000,2,3,5,3,40000,87800,3000000.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427392,2016,201601,353801,1521220,60,2016003538011,12,990712,1,2,...,3,960,2,3,5,3,3300,103300,158400.0,12
427393,2016,201601,353804,1521266,80,2016003538041,12,310712,1,1,...,6,2240,2,3,2,2,60700,60700,648000.0,0
427394,2016,201601,353805,1521273,88,2016003538051,12,860412,1,1,...,6,4350,2,3,5,3,426000,426000,37062000.0,12
427395,2016,201601,353812,1521439,63,2016003538121,12,1050212,1,1,...,6,2880,2,3,5,3,50001,50001,3200000.0,12


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,1142373,46,94,2010011423731,47,60047,1,2,...,6,5070,2,3,5,3,100000,124500,7800000.0,47
1,2010,201001,1142374,122,28,2010011423741,47,40047,1,1,...,6,972,2,3,5,3,29000,29000,270000.0,47
2,2010,201001,1142376,277,100,2010011423761,47,70047,1,1,...,6,4000,2,3,5,3,32000,75000,3200000.0,47
3,2010,201001,1142376,277,100,2010011423761,47,70047,1,2,...,4,3360,2,3,5,3,30000,75000,2520000.0,47
4,2010,201001,1142377,346,61,2010011423771,47,50247,1,1,...,6,3300,2,3,5,3,63000,67600,3780000.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152465,2016,201601,1152330,1521212,37,2016011523301,47,60047,1,1,...,6,1480,2,3,5,3,33000,33000,1221000.0,47
152466,2016,201601,1152332,1521295,33,2016011523321,47,250547,1,1,...,3,1190,2,3,5,3,16200,16200,380800.0,47
152467,2016,201601,1152333,1521327,125,2016011523331,47,30047,1,1,...,6,5000,2,3,5,3,35050,35050,4375000.0,47
152468,2016,201601,1152335,1521445,33,2016011523351,47,40047,1,1,...,6,1280,2,3,5,3,48004,181304,1536000.0,18


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,2,80,97,2010000000021,1,220001,1,1,...,6,1940,2,3,5,3,17000,30000,1649000.0,1
1,2010,201001,2,80,97,2010000000021,1,220001,1,2,...,6,2560,2,3,5,3,13000,30000,1664000.0,1
2,2010,201001,4,224,82,2010000000041,1,130001,1,1,...,6,3280,2,3,5,3,28000,73000,2296000.0,1
3,2010,201001,4,224,82,2010000000041,1,130001,1,2,...,6,3250,2,3,5,3,45000,73000,2925000.0,1
4,2010,201001,5,258,70,2010000000051,1,20001,1,1,...,6,2800,2,2,4,3,28000,28000,1960000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211045,2016,201601,1380226,1518675,24,2016013802261,56,40056,1,2,...,6,1485,2,3,5,3,77400,93900,1980000.0,56
2211046,2016,201601,1380227,1519445,65,2016013802271,56,50056,1,1,...,6,2925,2,3,5,3,65000,65000,4225000.0,56
2211047,2016,201601,1380228,1519624,73,2016013802281,56,40056,1,1,...,6,2190,2,3,5,3,22200,22200,1387000.0,56
2211048,2016,201601,1380229,1519920,67,2016013802291,56,40056,1,1,...,6,3350,2,3,5,3,75000,100000,5025000.0,56


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,911487,105,14,2010009114871,37,200037,1,1,...,6,490,2,3,5,3,17050,54550,238000.0,37
1,2010,201001,911487,105,14,2010009114871,37,200037,1,2,...,6,520,2,3,5,3,37500,54550,214500.0,37
2,2010,201001,911488,109,16,2010009114881,37,470037,1,2,...,6,560,2,1,5,3,50900,196900,509600.0,37
3,2010,201001,911489,119,52,2010009114891,37,20237,1,1,...,6,2340,2,1,2,1,80000,80000,1560000.0,0
4,2010,201001,911490,164,113,2010009114901,37,280137,1,1,...,6,5311,2,2,4,1,25000,32000,1695000.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233799,2016,201601,941231,1521145,91,2016009412311,37,70037,1,1,...,6,3640,2,3,5,3,35600,60500,3239600.0,51
233800,2016,201601,941232,1521204,7,2016009412321,37,50037,1,2,...,6,300,2,3,5,3,36000,77500,0.0,37
233801,2016,201601,941235,1521335,88,2016009412351,37,310837,1,1,...,4,1218,2,3,5,3,71700,114600,1479000.0,37
233802,2016,201601,941235,1521335,88,2016009412351,37,310837,1,2,...,6,3360,2,3,5,3,42900,114600,3603600.0,37


...

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,1367270,3,226,2010013672701,55,170055,1,1,...,6,10170,2,3,5,3,29000,29000,6554000.0,55
1,2010,201001,1367271,25,26,2010013672711,55,10055,1,2,...,6,920,2,3,5,3,60000,90000,1380000.0,17
2,2010,201001,1367272,151,79,2010013672721,55,150055,1,1,...,6,3120,2,3,5,3,24400,60600,1903200.0,55
3,2010,201001,1367272,151,79,2010013672721,55,150055,1,2,...,6,3300,2,3,5,3,36200,60600,2389200.0,55
4,2010,201001,1367276,221,134,2010013672761,55,120055,1,1,...,6,8040,2,3,5,3,35000,70000,4690000.0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157405,2016,201601,1377675,1520983,79,2016013776751,55,30055,1,2,...,6,3375,2,3,5,3,49800,127800,3735000.0,55
157406,2016,201601,1377678,1521054,256,2016013776781,55,2000055,1,1,...,6,9472,2,3,5,3,53000,75400,13568000.0,55
157407,2016,201601,1377679,1521056,52,2016013776791,55,70055,1,2,...,6,2950,2,3,5,3,48000,59300,2832000.0,17
157408,2016,201601,1377682,1521136,256,2016013776821,55,4030155,1,1,...,6,10240,2,3,5,3,59000,72000,15104000.0,55


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,745413,298,97,2010007454131,30,50030,1,1,...,6,3880,2,3,5,3,28000,28000,2716000.0,30
1,2010,201001,745414,511,94,2010007454141,30,70030,1,1,...,6,3720,2,3,4,3,25000,25000,2325000.0,30
2,2010,201001,745416,1127,94,2010007454161,30,40030,1,1,...,6,3760,2,1,5,3,30000,90000,2820000.0,30
3,2010,201001,745416,1127,94,2010007454161,30,40030,1,2,...,6,4360,2,3,5,3,60000,90000,6540000.0,30
4,2010,201001,745418,2229,29,2010007454181,30,50030,1,1,...,6,1680,2,1,5,1,80000,101300,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25168,2016,201601,739855,1519168,27,2016007398551,30,50030,1,1,...,6,1215,3,3,5,3,20000,20000,0.0,30
25169,2016,201601,739859,1519760,305,2016007398591,30,40030,1,1,...,6,16775,2,3,5,3,44400,92400,10980000.0,30
25170,2016,201601,739859,1519760,305,2016007398591,30,40030,1,2,...,6,9680,2,3,5,3,48000,92400,11616000.0,30
25171,2016,201601,739861,1520393,16,2016007398611,30,50030,1,1,...,6,960,2,3,5,3,100000,115000,1600000.0,30


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,1138390,407,35,2010011383901,46,40046,1,1,...,6,2040,2,3,5,3,20000,20000,0.0,46
1,2010,201001,1138391,874,262,2010011383911,46,10046,1,1,...,6,11835,2,1,4,1,313300,345300,73114000.0,46
2,2010,201001,1138391,874,262,2010011383911,46,10046,1,2,...,6,5880,2,1,4,1,32000,345300,0.0,46
3,2010,201001,1138392,1465,53,2010011383921,46,60046,1,1,...,6,2226,2,3,5,3,42000,56000,2226000.0,46
4,2010,201001,1138392,1465,53,2010011383921,46,60046,1,2,...,6,2356,2,3,5,3,14000,56000,0.0,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22558,2016,201601,1122706,1519610,29,2016011227061,46,50046,1,2,...,6,1200,2,3,5,3,60700,100100,885000.0,46
22559,2016,201601,1122707,1519617,40,2016011227071,46,50046,1,1,...,3,1600,1,2,4,3,70000,76000,1200000.0,0
22560,2016,201601,1122707,1519617,40,2016011227071,46,50046,1,2,...,6,272,2,3,5,3,6000,76000,204000.0,46
22561,2016,201601,1122709,1520582,144,2016011227091,46,30046,1,1,...,6,5800,2,3,5,3,43500,59340,6307500.0,46


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,512527,6,216,2010005125271,20,60220,1,1,...,6,7525,2,3,5,3,35000,35000,7525000.0,20
1,2010,201001,512529,395,58,2010005125291,20,30020,1,1,...,1,2320,1,2,4,1,4000,4000,232000.0,0
2,2010,201001,512532,536,211,2010005125321,20,140120,1,1,...,6,8400,2,3,5,3,50000,52000,10500000.0,20
3,2010,201001,512534,577,89,2010005125341,20,40020,1,1,...,6,3560,2,3,5,3,30500,47100,2714500.0,20
4,2010,201001,512534,577,89,2010005125341,20,40020,1,2,...,6,3570,2,3,5,3,16600,47100,1693200.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74807,2016,201601,521876,1520807,93,2016005218761,20,60420,1,2,...,6,3400,2,3,5,3,219000,236000,16915000.0,20
74808,2016,201601,521878,1521030,530,2016005218781,20,140020,1,1,...,6,15900,2,3,5,3,35200,67200,11660000.0,20
74809,2016,201601,521878,1521030,530,2016005218781,20,140020,1,2,...,6,12520,2,3,5,3,32000,67200,10016000.0,20
74810,2016,201601,521879,1521368,116,2016005218791,20,120020,1,1,...,1,1740,2,1,3,1,69000,69000,4060000.0,20


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,WKSWORK2,UHRSWORK,WRKLSTWK,LOOKING,AVAILBLE,WRKRECAL,INCTOT,FTOTINC,INCWAGE,PWSTATE2
0,2010,201001,1172058,19,287,2010011720581,48,350148,1,1,...,2,7722,2,3,5,3,1600,27600,457600.0,48
1,2010,201001,1172060,32,94,2010011720601,48,250948,1,1,...,6,3760,2,3,5,3,56000,56000,5264000.0,48
2,2010,201001,1172062,39,64,2010011720621,48,560548,1,1,...,6,2560,2,3,5,3,40300,40300,2566400.0,48
3,2010,201001,1172065,83,230,2010011720651,48,430048,1,1,...,3,9160,2,3,5,3,20000,20000,0.0,48
4,2010,201001,1172066,90,53,2010011720661,48,460848,1,1,...,6,2120,2,3,5,3,18000,18000,954000.0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598217,2016,201601,1260941,1521391,105,2016012609411,48,590948,1,2,...,6,5080,2,3,5,3,36000,51000,4572000.0,48
598218,2016,201601,1260942,1521392,44,2016012609421,48,130048,1,1,...,6,2200,2,3,5,3,68000,108000,2552000.0,48
598219,2016,201601,1260942,1521392,44,2016012609421,48,130048,1,2,...,6,2385,2,3,5,3,40000,108000,2120000.0,48
598220,2016,201601,1260946,1521423,81,2016012609461,48,350248,1,1,...,4,3280,2,3,5,3,52000,124000,4264000.0,48


In [37]:
# defining target
target = files_dict["MT"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]

In [38]:
target # view the target df

,HINSCAID,EMPSTAT,UHRSWORK,INCWAGE
0,1,1,3880,2716000.0
1,1,1,3720,2325000.0
2,1,1,3760,2820000.0
3,1,1,4360,6540000.0
4,1,1,1680,0.0
...,...,...,...,...
25168,1,1,1215,0.0
25169,1,1,16775,10980000.0
25170,1,1,9680,11616000.0
25171,1,1,960,1600000.0


In [62]:
# controls
control1 = files_dict["SC"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control2 = files_dict["FL"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control3 = files_dict["TN"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control4 = files_dict["NC"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control5 = files_dict["WY"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control6 = files_dict["GA"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control7 = files_dict["MS"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control8 = files_dict["AL"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control9 = files_dict["WI"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control10 = files_dict["SD"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control11 = files_dict["KS"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]
control12 = files_dict["TX"][["HINSCAID", "EMPSTAT", "UHRSWORK", "INCWAGE"]]

In [39]:
# code for implementation

## barycentric projection
def baryc_proj(source, target):
    
    n1 = source.shape[0]
    n2 = target.shape[0]   
    p = source.shape[1]
    a_ones, b_ones = np.ones((n1,)) / n1, np.ones((n2,)) / n2
    
    M = ot.dist(source, target)
    M = M.astype('float64')
    M /= M.max()
    OTplan = ot.bregman.sinkhorn_stabilized(a_ones, b_ones, M, reg = 5*1e-3)# try emd
    
    # initialization
    OTmap = np.empty((0, p))

    for i in range(n1):
        
        # normalization
        OTplan[i,:] = OTplan[i,:] / sum(OTplan[i,:])
    
        # conditional expectation
        OTmap = np.vstack([OTmap, (target.T @ OTplan[i,:])])
    
    OTmap = np.array(OTmap)
    
    return(OTmap)

In [40]:
# optimization routine

def to_optimize(lambdas):
    
    ans = []
    for i in range(J):
        temp = lambdas[i] * (G_list[i] - globtarget)
        ans.append(sum(sum(temp**2)))
    
    return sum(ans) / n

In [41]:
# optimal weights calculations

def get_optimal_weights(Glist):
        
    mylambda = cp.Variable(J)

    objective = cp.Minimize(to_optimize(mylambda))
    constraints = [mylambda >= 0, mylambda <= 1, cp.sum(mylambda) == 1]

    prob = cp.Problem(objective, constraints)
    prob.solve()

    weights = mylambda.value
    
    return(weights)

In [42]:
# full implementation

def DSCreplication(target, controls):
    
    global n, d, J, globtarget
    n = target.shape[0]
    d = target.shape[1]
    J = len(controls)
    globtarget = target

    # Barycentric Projection
    global G_list
    G_list = []
    for i in range(len(controls)):
        G_list.append(baryc_proj(target, controls[i]))
    
    
    # Obtain optimal weights(to_optimze needs to be pre-defined)
    weights = get_optimal_weights(G_list)
    projection = weights[0]*G_list[0]
    for j in range(J-1):
        projection += weights[j+1]*G_list[j+1]
    
    
    return(weights, projection)

def DSCreplicationV2(target, controls):
    
    n = target.shape[0]
    d = target.shape[1]
    J = len(controls)
    
    
    # Barycentric Projection
    G_list = []
    for i in range(len(controls)):
        G_list.append(baryc_proj(target, controls[i]))
    
    
    # Function to optimize
    def to_optimize(lambdas):
                
        ans = []
        for i in range(J):
            temp = lambdas[i] * (G_list[i] - target)
            ans.append(sum(sum(temp**2)))
    
        return sum(ans) / n

    
    # Obtain optimal weights
    mylambda = cp.Variable(J)

    objective = cp.Minimize(to_optimize(mylambda))
    constraints = [mylambda >= 0, mylambda <= 1, cp.sum(mylambda) == 1]

    prob = cp.Problem(objective, constraints)
    prob.solve()

    
    weights = mylambda.value
    projection = weights[0]*G_list[0]
    for j in range(J-1):
        projection += weights[j+1]*G_list[j+1]
    
    
    return(weights, projection)

In [ ]:
# testing below

In [45]:
n1 = n2 = 100
dim = 3

covmat = np.zeros((dim, dim))
np.fill_diagonal(covmat, 1)

covmat2 = np.full((dim, dim), 0.3)
np.fill_diagonal(covmat2, 1)

covmat3 = np.full((dim, dim), 0.5)
np.fill_diagonal(covmat3, 1)

covmat4 = np.full((dim, dim), 0.8)
np.fill_diagonal(covmat4, 1)

a_ones, b_ones = np.ones((n1,)) / n1, np.ones((n2,)) / n2

c1 = random.multivariate_normal(mean = [20, 20, 20], cov = covmat4, size = n1)
c2 = random.multivariate_normal(mean = [100, 100, 100], cov = covmat4, size = n1)
c3 = random.multivariate_normal(mean = [50, 50, 50], cov = covmat3, size = n1)
cs = [c1, c2, c3]

TARGET = random.multivariate_normal(mean = [25]*dim, cov = covmat, size = n1)

In [60]:
type(cs[1])

numpy.ndarray

In [57]:
type(TARGET)

numpy.ndarray

In [46]:
ts = t.time()

weights1, projection1 = DSCreplication(TARGET, cs)

print(t.time() - ts)
print(weights1)

3.301009178161621
[0.95462633 0.00453176 0.0408419 ]


In [48]:
# full implementation and estimation

In [64]:
states_controls = [control1.to_numpy(), control2.to_numpy()]
type(states_controls)

list

In [67]:
target = target.to_numpy()

In [ ]:
weights_s, projection_s = DSCreplicationV2(target, states_controls)